In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gseapy as gp

In [121]:
enriched_raw = pd.read_csv('./enrichemnt.csv')
enriched_raw['RNA-control.bam'] = enriched_raw['RNA-control.bam'].replace(0,1)
enriched_raw['clip_enrichment'] = enriched_raw['CLIP-35L33G.bam'] / enriched_raw['RNA-control.bam']

enriched = enriched_raw[['gene_id','clip_enrichment']]
enriched = enriched[enriched['clip_enrichment'] > 1]
enriched['log2ce'] = np.log2(enriched['clip_enrichment'])
enriched.head()

,gene_id,clip_enrichment,log2ce
2,ENSMUSG00000051951,4.0,2.000000
3,ENSMUSG00000102851,3.0,1.584963
8,ENSMUSG00000103201,6.0,2.584963
10,ENSMUSG00000103161,17.0,4.087463
11,ENSMUSG00000102331,2.0,1.000000


In [122]:
highenriched = enriched[enriched['log2ce'] > 2].sort_values(by='log2ce', ascending=False)
highenriched_ids = highenriched['gene_id'].tolist()
print(len(highenriched_ids))
highenriched_ids[:5]

8180


['ENSMUSG00000105925',
 'ENSMUSG00000064899',
 'ENSMUSG00000064336',
 'ENSMUSG00000118717',
 'ENSMUSG00000064349']

### Convert Ensembl ID to Gene Symbol
> At this point, used g:Convert instead of biomart because biomart is not working properly.
1. save enriched genes as a list
2. convert ensembl id to gene symbol using g:Convert
   - **8180** ensembl ids are converted to **8178** unique gene symbols

### Enrichment Analysis
- Tools
  - g:Profiler w/ ensembl id
  - enrichr w/ gene symbol

In [123]:
with open ('./enriched_ensembl.txt', 'w') as f:
    for id in highenriched_ids:
        f.write(id + '\n')

In [115]:
## import converted gene symbols
ensembl_symbol = pd.read_csv('./enriched_ensembl_symbol.csv', names=['gene_id', 'gene_symbol'], header=0)
gene_symbol = pd.merge(highenriched, ensembl_symbol, on='gene_id')
gene_symbol.head()

,gene_id,clip_enrichment,log2ce,gene_symbol
0,ENSMUSG00000105925,3113.000000,11.604090,Gm43846
1,ENSMUSG00000064899,2667.000000,11.381002,Snord118
2,ENSMUSG00000064349,1060.916667,10.051096,mt-Tc
3,ENSMUSG00000118876,882.000000,9.784635,Rnu1b1
4,ENSMUSG00000069266,844.000000,9.721099,H4c2


In [116]:
names = gp.get_library_name('Mouse')
gene_list = gene_symbol['gene_symbol'].tolist()


In [62]:
names

['ARCHS4_Cell-lines',
 'ARCHS4_IDG_Coexp',
 'ARCHS4_Kinases_Coexp',
 'ARCHS4_TFs_Coexp',
 'ARCHS4_Tissues',
 'Achilles_fitness_decrease',
 'Achilles_fitness_increase',
 'Aging_Perturbations_from_GEO_down',
 'Aging_Perturbations_from_GEO_up',
 'Allen_Brain_Atlas_10x_scRNA_2021',
 'Allen_Brain_Atlas_down',
 'Allen_Brain_Atlas_up',
 'Azimuth_Cell_Types_2021',
 'BioCarta_2013',
 'BioCarta_2015',
 'BioCarta_2016',
 'BioPlanet_2019',
 'BioPlex_2017',
 'CCLE_Proteomics_2020',
 'CORUM',
 'COVID-19_Related_Gene_Sets',
 'COVID-19_Related_Gene_Sets_2021',
 'Cancer_Cell_Line_Encyclopedia',
 'CellMarker_Augmented_2021',
 'ChEA_2013',
 'ChEA_2015',
 'ChEA_2016',
 'ChEA_2022',
 'Chromosome_Location',
 'Chromosome_Location_hg19',
 'ClinVar_2019',
 'DSigDB',
 'Data_Acquisition_Method_Most_Popular_Genes',
 'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019',
 'DepMap_WG_CRISPR_Screens_Sanger_CellLines_2019',
 'Descartes_Cell_Types_and_Tissue_2021',
 'Diabetes_Perturbations_GEO_2022',
 'DisGeNET',
 'Disease_

In [118]:
gene_sets = ['GO_Biological_Process_2023','GO_Cellular_Component_2023','GO_Molecular_Function_2023', 'Mouse_Gene_Atlas', 'Allen_Brain_Atlas_10x_scRNA_2021']
go_bp = gp.enrichr(gene_list=gene_list,
                 gene_sets=gene_sets,
                 organism='Mouse', outdir=None, cutoff=0.05)

In [68]:
from gseapy import barplot, dotplot

ax = dotplot(enr.res2d,title='GO_Biological_Process_2023', x='Gene_set', column='Adjusted P-value'
             figsize=(10,6),cmap='viridis_r',dot_min=10)

                        Gene_set  \
0     GO_Biological_Process_2023   
1     GO_Biological_Process_2023   
2     GO_Biological_Process_2023   
3     GO_Biological_Process_2023   
4     GO_Biological_Process_2023   
...                          ...   
6316            Mouse_Gene_Atlas   
6317            Mouse_Gene_Atlas   
6318            Mouse_Gene_Atlas   
6319            Mouse_Gene_Atlas   
6320            Mouse_Gene_Atlas   

                                                   Term Overlap   P-value  \
0     Dermatan Sulfate Proteoglycan Metabolic Proces...     5/5  0.039168   
1                       Insulin Processing (GO:0030070)     5/5  0.039168   
2                Neurotransmitter Reuptake (GO:0098810)     7/8  0.046491   
3                         Serotonin Uptake (GO:0051610)     7/8  0.046491   
4     Dermatan Sulfate Proteoglycan Biosynthetic Pro...     6/7  0.079116   
...                                                 ...     ...       ...   
6316                        